# Classification task

In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt
import collections
import pydotplus 
import statistics 
import pandas as pd
import os
from datetime import date
from tqdm.notebook import tqdm
from IPython.display import Image  
import scikitplot as skplt
import wittgenstein as lw

from scipy.stats.stats import pearsonr
from scipy.spatial.distance import pdist,  squareform
import scipy.stats as stats
from scipy.stats import randint as sp_randint

from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score, pairwise_distances, classification_report, confusion_matrix, plot_confusion_matrix # For Model evaluation
from sklearn.metrics import make_scorer, accuracy_score
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split, cross_validate, RandomizedSearchCV, GridSearchCV
from sklearn import tree, metrics
from sklearn.datasets import make_blobs

import seaborn as sns
import re

In [ ]:
# load of the data
DATASET_DIR = "dataset" + os.path.sep
#index_col=False say to not use the first column as ID
df_players = pd.read_csv('players.csv', sep=',', index_col=0) 

In [ ]:
df_players.info()

In [ ]:
df_players[['sex', 'hand', 'best_rank','best_rank_points', 'best_of_3_match', 'best_of_5_match',
           'best_of_3_wins', 'best_of_5_wins', 'w_finals', 'tot_minutes', 'sv1st', 'sv1st_win', 'sv2nd_win', 
           'df', 'ace_perc', 'bpS_perc', 'wmatch', 'lmatch', 'nmatch', 'n_tourney']]

In [ ]:
def discretize_data(dataset, variables): #mapping categorical into numerical
    for variable in variables:
        #get the unique variable's values
        var = sorted(dataset[variable].unique())
        
        #generate a mapping from the variable's values to the number representation  
        mapping = dict(zip(var, range(0, len(var) + 1)))

        #add a new colum with the number representation of the variable
        dataset[variable+'_num'] = dataset[variable].map(mapping).astype(int)
    return dataset

In [ ]:
df_p = df_players[['sex', 'hand', 'best_rank','best_rank_points', 'best_of_3_match', 'best_of_5_match',
           'best_of_3_wins', 'best_of_5_wins', 'w_finals', 'tot_minutes', 'sv1st', 'sv1st_win', 'sv2nd_win', 
           'df', 'ace_perc', 'bpS_perc', 'wmatch', 'lmatch', 'nmatch', 'n_tourney']]

In [ ]:
variables = ['sex', 'hand']
df_p = discretize_data(df_p, variables)

In [ ]:
df_p.drop(columns=['sex', 'hand'], inplace=True, axis=1)

In [ ]:
df_p

In [ ]:
threshold = 50
df_p.loc[((df_p['best_rank']>0) & (df_p['best_rank']<=threshold)), 'ranked'] = 'high'
df_p.loc[((df_p['best_rank']>0) & (df_p['best_rank']>threshold)), 'ranked'] = 'low'

In [ ]:
df_classif = df_p[df_p['best_rank']>0]

In [ ]:
df_classif.drop(columns=['best_rank'], inplace=True, axis=1)

In [ ]:
label = df_classif.pop('ranked')
train_set, test_set, train_label, test_label = train_test_split(df_classif, label, stratify =label, test_size=0.40)

### Decision tree

In [ ]:
train_set

In [ ]:
dt = tree.DecisionTreeClassifier(criterion='gini', splitter='best', 
                                  max_depth=5, 
                                  min_samples_split=3, min_samples_leaf=4)
dt = dt.fit(train_set, train_label)

In [ ]:
dot_data = tree.export_graphviz(dt, out_file=None, 
                         feature_names=list(train_set.columns),  
                         class_names=['high', 'low'],  
                         filled=True, rounded=True)  
graph = pydotplus.graph_from_dot_data(dot_data)  
Image(graph.create_png())

In [ ]:
train_pred_dt = dt.predict(train_set)
test_pred_dt = dt.predict(test_set)

#### evaluation

In [ ]:
dt.predict_proba(train_set)

In [ ]:
#evaulate the accuracy on the train set and the test set
#metrics also contains precision, recall, f1 and the support
print('Accuracy train set ', metrics.accuracy_score(train_label, train_pred_dt))
print('Accuracy test set ', metrics.accuracy_score(test_label, test_pred_dt))
print('Precision train set ', metrics.precision_score(train_label, train_pred_dt, average='weighted'))
print('Recall train set ', metrics.recall_score(train_label, train_pred_dt, average='weighted'))
print('F1 score train set ', metrics.f1_score(train_label, train_pred_dt, average='weighted'))
print('Support train set ', metrics.precision_recall_fscore_support(train_label, train_pred_dt))

In [ ]:
#metrics computed on the test set
def report_scores(test_label, test_pred):
    print(classification_report(test_label, 
                            test_pred, 
                            target_names=['low', 'high']))

In [ ]:
#per il training set
report_scores(train_label, train_pred_dt)

In [ ]:
#per il test set
report_scores(test_label, test_pred_dt)
#l'accuracy è un buon indicatore, è significativa se è maggiore dell'accuracy della majority class. in caso di 
#situazione unbalance anche la precision e la recall aiutano a capire quanti errori abbiamo

In [ ]:
### cross validation

In [ ]:
scores = cross_validate(dt, train_set, train_label, cv=3, return_train_score= True)
print('Fit time ', statistics.mean(scores['fit_time']))
print('Score time ', statistics.mean(scores['score_time']))
print('Test score ', statistics.mean(scores['test_score']))
print('Train score ', statistics.mean(scores['train_score']))

In [ ]:
#compute confusion matrix
cm = confusion_matrix(test_label, test_pred_dt)
cm

In [ ]:
#it is possible to plot the confusion matrix 
plot_confusion_matrix(dt, test_set, test_label)
plt.show() 

In [ ]:
#true labels - different colors for different class
color = ['red' if label=='low'  else 'green' for label in test_label]
plt.scatter(test_set.iloc[:, 5].values, test_set.iloc[:, 2].values , c=color, s=25);

#### excluded raws

In [ ]:
df_to_rank = df_p[df_p['best_rank']==0]
df_to_rank.drop(columns=['best_rank'], inplace=True, axis=1)
df_to_rank.drop(columns = ['ranked'], inplace = True, axis=1)

In [ ]:
unknown_pred_dt = dt.predict(df_to_rank)

In [ ]:
df_to_rank['ranked'] = unknown_pred_dt

In [ ]:
df_to_rank

### SVM

In [ ]:
svm = SVC(kernel='sigmoid', C=0.5, gamma='scale', probability=True)
svm.fit(train_set, train_label)

In [ ]:
#prediction on the test test
test_pred_proba_svm = svm.predict_proba(test_set)
test_pred_proba_svm

In [ ]:
test_pred_svm = svm.predict(test_set)

In [ ]:
#compute the performance of the model
print(classification_report(test_label, 
                            test_pred_svm, 
                            target_names=['low', 'high']))

In [ ]:
test_pred_proba = svm.predict_proba(test_set)
skplt.metrics.plot_roc_curve(test_label.values, test_pred_proba) #roc curve for only one model
plt.show()

### Rule based

In [ ]:
df_classif = df_p[df_p['best_rank']>0]

In [ ]:
df_classif.drop(columns=['ranked'], inplace=True, axis=1)

In [ ]:
df_classif.loc[((df_classif['best_rank']>0) & (df_classif['best_rank']<=threshold)), 'ranked'] = True #low
df_classif.loc[((df_classif['best_rank']>0) & (df_classif['best_rank']>threshold)), 'ranked'] = False #high

In [ ]:
df_classif.drop(columns=['best_rank'], inplace=True, axis=1)

In [ ]:
df_classif

In [ ]:
label = df_classif.pop('ranked')
train_set, test_set, train_label, test_label = train_test_split(df_classif, label, stratify =label, test_size=0.40)

In [ ]:
#we run a grid search to find the best configuration of parameters' values
ripper = lw.RIPPER()
param_grid = {"prune_size": [0.5, 0.6], "k": [1, 3, 5]}
grid_search = GridSearchCV(estimator=ripper, param_grid=param_grid)
grid_search.fit(train_set, train_label, pos_class=1)

In [ ]:
print('Best parameters setting ', grid_search.cv_results_['params'][0])

In [ ]:
#define and fit the rule-based model
#this function requires only one dataset with the labels. 
#To do so, we concatenate the train_set and the train_label
ripper = lw.RIPPER(k=1, prune_size=0.50)
datas = pd.concat([train_set, train_label], axis=1)
ripper.fit(datas, class_feat='ranked', pos_class=1)

In [ ]:
#in this case the model is a set of rules
ripper.out_model()

In [ ]:
test_set.head(34)

In [ ]:
ripper.predict(test_set)

In [ ]:
#evaluation of the performance of the classifier
from sklearn.metrics import precision_score, recall_score
print('Accuracy ', ripper.score(test_set, test_label))
print('Precision ', ripper.score(test_set, test_label, precision_score))
print('Recall ', ripper.score(test_set, test_label, recall_score))

In [ ]:
#with rule based classifiers it is possible to extract the reasons for the prediction.
#only for the positive predictions
ripper.predict(test_set[:15], give_reasons=True) #give_reasons dice quali regole sono vere

### Naive Bayes

### Bagging

### Boosting

### AdaBoost

### Random Forest

### Neural Networks

### KNN